In [1]:
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

map_dirs = ['map 3', 'map 4', 'map 5']
# map_dirs = ['map 4']

paths = {'map 3': './data/map 3/data-of-baselines-ddqn-sarl.json',
        'map 4': './data/map 4/data-of-baselines.json',
        'map 5': './data/map 5/data-of-baselines-ddqn-sarl-new.json'}


# Combine all maps together:

In [2]:
participants_df = pd.DataFrame()
raw_df_state_to_action = pd.DataFrame()
for map_dir in map_dirs:
    path = paths[map_dir]
    #load data from the json file
    with open(path) as train_file:
        data = json.load(train_file)

    participants_df_temp = pd.DataFrame.from_dict(data['all-games'], orient='index')

    path = 'data/'+map_dir+'/workers' # use your path
    all_files = glob.glob(path + "/*.csv")

    li = []
    for filename in all_files:
        fromOne = pd.read_csv(filename, index_col=None, header=0)
        fromOne['filename'] = filename
        li.append(fromOne)

    workers = pd.concat(li, axis=0, ignore_index=True)
    workers = workers.set_index('Answer.surveycode')
    workers.index = workers.index.map(lambda code: '-' + code[:-3])
    # print(workers.index[workers.index.duplicated()])
    workers = workers[~workers.index.duplicated(keep='first')]
    participants_df_temp['WorkerId'] = workers['WorkerId']
    participants_df_temp = participants_df_temp.dropna(subset = ['WorkerId'])

    if map_dir == 'map 4':
        # duplicate selfish to ddqn and sarl
        selfish_df = participants_df_temp[participants_df_temp['behavior'] == 'selfish']
        ddqn_df = selfish_df.copy()
        ddqn_df['behavior'] = 'ddqn distribution'
        sarl_df = selfish_df.copy()
        sarl_df['behavior'] = 'sarl ddqn distribution'
        participants_df_temp = pd.concat([participants_df_temp, ddqn_df, sarl_df])

    participants_df = pd.concat([participants_df, participants_df_temp])

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
index_to_drop = participants_df[participants_df.duplicated('WorkerId', keep='first')].index
print(len(index_to_drop))

participants_df = participants_df[~participants_df.duplicated('WorkerId', keep='first')]

514


number of people that answer the survay only once:

In [4]:
len(participants_df)

369

In [5]:
participants_df.loc[: ,'human_score'] = pd.to_numeric(participants_df['human_score'], errors='coerce')
participants_df.loc[: ,'computer_score'] = pd.to_numeric(participants_df['computer_score'], errors='coerce')
participants_df.loc[: ,'collaborative_value'] = pd.to_numeric(participants_df['collaborative_value'], errors='coerce')
participants_df.loc[: ,'predictable_value'] = pd.to_numeric(participants_df['predictable_value'], errors='coerce')
participants_df.loc[: ,'selfishly_value'] = pd.to_numeric(participants_df['selfishly_value'], errors='coerce')
participants_df.loc[: ,'wisely_value'] = pd.to_numeric(participants_df['wisely_value'], errors='coerce')
participants_df.loc[: ,'computer_value'] = pd.to_numeric(participants_df['computer_value'], errors='coerce')

In [6]:
participants_df.groupby('behavior')[['computer_score', 'human_score', 'collaborative_value', 'predictable_value', 'selfishly_value', 'wisely_value', 'computer_value']].mean()

,computer_score,human_score,collaborative_value,predictable_value,selfishly_value,wisely_value,computer_value
behavior,,,,,,,
TSP,0.822727,0.878333,5.212121,5.121212,2.863636,5.378788,5.575758
closest,0.741791,0.825821,4.850746,5.059701,3.298507,4.746269,6.000000
ddqn distribution,1.029667,-0.065333,2.500000,5.266667,6.066667,3.533333,5.233333
farthest,0.449672,0.510164,3.803279,4.344262,4.393443,3.819672,5.475410
random,0.257581,0.262903,2.516129,4.080645,5.403226,3.032258,5.919355
sarl ddqn distribution,1.175714,0.740000,3.714286,4.571429,5.000000,4.914286,5.600000
selfish,1.078333,0.076667,2.520833,5.375000,6.208333,3.791667,5.854167


In [7]:
participants_df['behavior'].value_counts()

closest                   67
TSP                       66
random                    62
farthest                  61
selfish                   48
sarl ddqn distribution    35
ddqn distribution         30
Name: behavior, dtype: int64

In [8]:
participants_df['gender'].value_counts()

male      226
female    143
Name: gender, dtype: int64

In [9]:
participants_df['education'].value_counts()

BA                 194
High School        105
Master's degree     66
Ph.D                 4
Name: education, dtype: int64

# Build stay table

In [10]:
c_h_stays = pd.DataFrame()
for map_dir in map_dirs:
    path = paths[map_dir]
    #load data from the json file
    with open(path) as train_file:
        data = json.load(train_file)
    
    raw_df_state_to_action_temp = pd.DataFrame.from_dict(data['humanModel'], orient='index')
    raw_df_state_to_action_temp = raw_df_state_to_action_temp.drop(0, axis=1)

    participants_df_temp = pd.DataFrame.from_dict(data['all-games'], orient='index')

    path = 'data/'+map_dir+'/workers' # use your path
    all_files = glob.glob(path + "/*.csv")

    li = []
    for filename in all_files:
        fromOne = pd.read_csv(filename, index_col=None, header=0)
        fromOne['filename'] = filename
        li.append(fromOne)

    workers = pd.concat(li, axis=0, ignore_index=True)
    workers = workers.set_index('Answer.surveycode')
    workers.index = workers.index.map(lambda code: '-' + code[:-3])
    # print(workers.index[workers.index.duplicated()])
    workers = workers[~workers.index.duplicated(keep='first')]
    participants_df_temp['WorkerId'] = workers['WorkerId']
    participants_df_temp = participants_df_temp.dropna(subset = ['WorkerId'])

    if map_dir == 'map 4':
        # duplicate selfish to ddqn and sarl
        selfish_df = participants_df_temp[participants_df_temp['behavior'] == 'selfish']
        ddqn_df = selfish_df.copy()
        ddqn_df['behavior'] = 'ddqn distribution'
        sarl_df = selfish_df.copy()
        sarl_df['behavior'] = 'sarl ddqn distribution'
        participants_df_temp = pd.concat([participants_df_temp, ddqn_df, sarl_df])

    index_to_drop = participants_df_temp[participants_df_temp.duplicated('WorkerId', keep='first')].index
    print(len(index_to_drop))
    
    participants_df_temp = participants_df_temp[~participants_df_temp.duplicated('WorkerId', keep='first')]

    null_index = raw_df_state_to_action_temp.index.difference(participants_df_temp.index, sort=False)
    raw_df_state_to_action_finish_game = raw_df_state_to_action_temp.drop(null_index)

    participants_df_temp = participants_df_temp.reset_index()

    def extractState(cell):
        if cell == None:
            return cell
        if cell != cell:
            return cell
        board = np.array(cell['state'][0]).astype(float)
        human_trace = np.array(cell['state'][1]).astype(float)
        computer_trace = np.array(cell['state'][2]).astype(float)
        human_awards = np.array(cell['state'][3]).astype(float)
        computer_awards = np.array(cell['state'][4]).astype(float)
        all_awards = np.array(cell['state'][5]).astype(float)
        return (board, human_trace, computer_trace, human_awards, computer_awards, all_awards)

    state_dim_6_temp = pd.DataFrame(columns=raw_df_state_to_action_finish_game.columns)
    for col in raw_df_state_to_action_finish_game:
        state_dim_6_temp[col] = raw_df_state_to_action_finish_game[col].apply(extractState)

    state_dim_6_temp[len(state_dim_6_temp.columns) + 1] = None

    state_dim_6_temp = state_dim_6_temp.reset_index()

    def countStays(row, agent):
        stay_count = 0
        for i in range(2, len(row)+1):
            if row[i] != None: # game not ended
                if agent == 'computer':
                    trace_agent = row[i][2]
                else: # agent == 'human'
                    trace_agent = row[i][1]
                if 0.9 not in trace_agent:
                    stay_count += 1
            else:
                return stay_count
    
    c_h_stays_temp = pd.DataFrame(index=state_dim_6_temp.index, columns=[])
    computer_stays = []
    human_stays = []
    for i, row in state_dim_6_temp.iterrows():
        computer_num_of_stays = countStays(row, 'computer')
        human_num_of_stays = countStays(row, 'human')

        computer_stays.append(computer_num_of_stays)
        human_stays.append(human_num_of_stays)

    c_h_stays_temp['computer_stays'] = computer_stays
    c_h_stays_temp['human_stays'] = human_stays
    c_h_stays_temp['behavior'] = participants_df_temp['behavior']
    c_h_stays_temp['gender'] = participants_df_temp['gender']
    c_h_stays_temp['education'] = participants_df_temp['education']

    c_h_stays = pd.concat([c_h_stays, c_h_stays_temp])
    # c_h_stays.index.name = ''
    # c_h_stays = c_h_stays.reset_index()

89
192
190


In [11]:
c_h_stays

,computer_stays,human_stays,behavior,gender,education
0,2,0,closest,male,High School
1,160,131,selfish,female,BA
2,0,0,TSP,male,BA
3,2,0,farthest,female,Master's degree
4,1,1,farthest,male,High School
...,...,...,...,...,...
136,3,2,farthest,male,BA
137,14,2,random,female,High School
138,19,1,random,male,BA
139,1,0,farthest,male,BA


#### Average number of stays per agent:

In [12]:
c_h_stays.groupby('behavior').mean()

,computer_stays,human_stays
behavior,,
TSP,0.944444,2.611111
closest,0.887324,3.112676
ddqn distribution,42.366667,8.800000
farthest,3.930556,2.305556
random,9.819444,4.930556
sarl ddqn distribution,13.210526,1.210526
selfish,41.210526,9.105263


#### Average number of stays by education:

In [13]:
c_h_stays[c_h_stays['behavior'] == 'selfish'].groupby('education').mean()

,computer_stays,human_stays
education,,
BA,44.055556,12.250000
High School,30.444444,1.333333
Master's degree,40.750000,5.500000


#### Average score by education:

In [14]:
participants_df[participants_df['behavior'] == 'selfish'][['computer_score', 'human_score','education']].groupby('education').mean()

,computer_score,human_score
education,,
BA,1.049677,0.071290
High School,1.186250,0.245000
Master's degree,1.081111,-0.054444


#### Average number of stays by gender:

In [15]:
c_h_stays[c_h_stays['behavior'] == 'selfish'].groupby('gender').mean()

,computer_stays,human_stays
gender,,
female,42.384615,8.653846
male,40.225806,9.483871


#### Average score by gender:

In [16]:
participants_df[participants_df['behavior'] == 'selfish'][['computer_score', 'human_score','gender']].groupby('gender').mean()

,computer_score,human_score
gender,,
female,1.060952,-0.015238
male,1.091852,0.148148


## Agents score divided by gender

#### Per agent:

In [17]:
participants_df.groupby(['behavior', 'gender']).mean()[['computer_score', 'human_score']]

computer_score  human_score
behavior               gender                             
TSP                    female        0.822857     0.863429
                       male          0.822581     0.895161
closest                female        0.752500     0.758000
                       male          0.737234     0.854681
ddqn distribution      female        0.972500    -0.075833
                       male          1.067778    -0.058333
farthest               female        0.417000     0.487000
                       male          0.465610     0.521463
random                 female        0.286364     0.183182
                       male          0.241750     0.306750
sarl ddqn distribution female        1.168462     0.652308
                       male          1.180000     0.791818
selfish                female        1.060952    -0.015238
                       male          1.091852     0.148148

## Agents score divided by education:

#### Per agent:

In [18]:
participants_df.groupby(['behavior', 'education']).mean()[['computer_score', 'human_score']]

computer_score  human_score
behavior               education                                   
TSP                    BA                     0.820000     0.903000
                       High School            0.894000     0.930000
                       Master's degree        0.690000     0.728000
                       Ph.D                   0.450000     0.350000
closest                BA                     0.741176     0.780294
                       High School            0.779412     0.892353
                       Master's degree        0.703125     0.851875
ddqn distribution      BA                     1.035333    -0.144000
                       High School            1.118333     0.100000
                       Master's degree        0.961111    -0.044444
farthest               BA                     0.523000     0.582000
                       High School            0.363000     0.478500
                       Master's degree        0.368000     0.320000
                       Ph.D                   0.800000     0.890000
random                 BA                     0.252941     0.298235
                       High School            0.326471     0.247059
                       Master's degree        0.097000     0.126000
                       Ph.D                   0.850000     0.700000
sarl ddqn distribution BA                     1.191500     0.640000
                       High School            1.180000     0.887500
                       Master's degree        0.980000     0.700000
                       Ph.D                   1.200000     1.050000
selfish                BA                     1.049677     0.071290
                       High School            1.186250     0.245000
                       Master's degree        1.081111    -0.054444